You can read an overview of this Numerical Linear Algebra course in [this blog post](http://www.fast.ai/2017/07/17/num-lin-alg/).  The course was originally taught in the [University of San Francisco MS in Analytics](https://www.usfca.edu/arts-sciences/graduate-programs/analytics) graduate program.  Course lecture videos are [available on YouTube](https://www.youtube.com/playlist?list=PLtmWHNX-gukIc92m1K0P6bIOnZb-mg0hY) (note that the notebook numbers and video numbers do not line up, since some notebooks took longer than 1 video to cover).

You can ask questions about the course on [our fast.ai forums](http://forums.fast.ai/c/lin-alg).

# 7. PageRank with Eigen Decompositions

### Two Handy Tricks

Here are two tools that we'll be using today, which are useful in general.

1\. [Psutil](https://github.com/giampaolo/psutil) is a great way to check on your memory usage.  This will be useful here since we are using a larger data set.

In [1]:
import psutil
import os

In [11]:
process = psutil.Process(os.getpid())
t = process.memory_info()

In [12]:
t.vms, t.rss

(288772096, 60108800)

In [2]:
def mem_usage():
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / psutil.virtual_memory().total

In [14]:
mem_usage()

0.003542925463468014

2\. [TQDM](https://github.com/tqdm/tqdm) gives you progress bars.

In [3]:
from time import sleep

In [16]:
# Without TQDM
s = 0
for i in range(10):
    s += i
    sleep(0.2)
print(s)

45


In [17]:
# With TQDM
from tqdm import tqdm

s = 0
for i in tqdm(range(10)):
    s += i
    sleep(0.2)
print(s)

100%|██████████| 10/10 [00:02<00:00,  4.94it/s]

45


## Motivation

**Review**
- What is SVD?
- What are some applications of SVD?

#### Additional SVD Application

An interesting use of SVD that I recently came across was as a step in the de-biasing of Word2Vec word embeddings, from [Quantifying and Reducing Stereotypes in Word Embeddings](https://arxiv.org/pdf/1606.06121.pdf)(Bolukbasi, et al).

Word2Vec is a useful library released by Google that represents words as vectors.  The similarity of vectors captures semantic meaning, and analogies can be found, such as *Paris:France :: Tokyo: Japan*.  

<img src="images/word2vec_analogies.png" alt="" style="width: 80%"/>
(source: [Vector Representations of Words](https://www.tensorflow.org/versions/r0.10/tutorials/word2vec/))

However, these embeddings can implicitly encode bias, such as *father:doctor :: mother:nurse* and *man:computer programmer :: woman:homemaker*.

One approach for de-biasing the space involves using SVD to reduce the dimensionality ([Bolukbasi paper](https://arxiv.org/pdf/1606.06121.pdf)).

You can read more about bias in word embeddings:
- [How Vector Space Mathematics Reveals the Hidden Sexism in Language](https://www.technologyreview.com/s/602025/how-vector-space-mathematics-reveals-the-hidden-sexism-in-language/)(MIT Tech Review)
- [ConceptNet: better, less-stereotyped word vectors](https://blog.conceptnet.io/2017/04/24/conceptnet-numberbatch-17-04-better-less-stereotyped-word-vectors/)
- [Semantics derived automatically from language corpora necessarily contain human biases](https://www.princeton.edu/~aylinc/papers/caliskan-islam_semantics.pdf) (excellent and very interesting paper!)

#### Ways to think about SVD

- Data compression
- SVD trades a large number of features for a smaller set of better features
- All matrices are diagonal (if you use change of bases on the domain and range)

### Perspectives on SVD

We usually talk about SVD in terms of matrices, $$A = U \Sigma V^T$$ but we can also think about it in terms of vectors.  SVD gives us sets of orthonormal vectors ${v_j}$ and ${u_j}$ such that $$ A v_j = \sigma_j u_j $$

$\sigma_j$ are scalars, called singular values

**Q**: Does this remind you of anything?

#### Answer

**Relationship between SVD and Eigen Decomposition**: the left-singular vectors of A are the eigenvectors of $AA^T$. The right-singular vectors of A are the eigenvectors of $A^T A$. The non-zero singular values of A are the square roots of the eigenvalues of $A^T A$ (and $A A^T$).

SVD is a generalization of eigen decomposition. Not all matrices have eigen values, but ALL matrices have singular values.

Let's forget SVD for a bit and talk about how to find the eigenvalues of a symmetric positive definite matrix...

### Further resources on SVD

- [SVD: image compression and least squares](http://andrew.gibiansky.com/blog/mathematics/cool-linear-algebra-singular-value-decomposition/)

- [Image Compression with SVD](http://nbviewer.jupyter.org/gist/frankcleary/4d2bd178708503b556b0)

- [The Extraordinary SVD](https://sites.math.washington.edu/~morrow/498_13/svd_applied.pdf)

- [A Singularly Valuable Decomposition: The SVD of a Matrix](https://sites.math.washington.edu/~morrow/498_13/svd.pdf)

## Today: Eigen Decomposition

The best classical methods for computing the SVD are variants on methods for computing eigenvalues.  In addition to their links to SVD, Eigen decompositions are useful on their own as well.  Here are a few practical applications of eigen decomposition:
- [rapid matrix powers](http://www.onmyphd.com/?p=eigen.decomposition#h2_why)
- [nth Fibonacci number](http://mathproofs.blogspot.com/2005/04/nth-term-of-fibonacci-sequence.html)
- Behavior of ODEs
- Markov Chains (health care economics, Page Rank)
- [Linear Discriminant Analysis on Iris dataset](http://sebastianraschka.com/Articles/2014_python_lda.html)

Check out the 3 Blue 1 Brown videos on [Change of basis](https://www.youtube.com/watch?v=P2LTAUO1TdA) and [Eigenvalues and eigenvectors](https://www.youtube.com/watch?v=PFDu9oVAE-g)

"Eigenvalues are a way to see into the heart of a matrix... All the difficulties of matrices are swept away" -Strang 

**Vocab**: A **Hermitian** matrix is one that is equal to it's own conjugate transpose.  In the case of real-valued matrices (which is all we are considering in this course), **Hermitian** means the same as **Symmetric**.

**Relevant Theorems:**
- If A is symmetric, then eigenvalues of A are real and $A = Q \Lambda Q^T$
- If A is triangular, then its eigenvalues are equal to its diagonal entries

## DBpedia Dataset

Let's start with the **Power Method**, which finds one eigenvector.  *What good is just one eigenvector?* you may be wondering.  This is actually the basis for PageRank (read [The $25,000,000,000 Eigenvector: the Linear Algebra Behind Google](http://www.rose-hulman.edu/~bryan/googleFinalVersionFixed.pdf) for more info)

Instead of trying to rank the importance of all websites on the internet, we are going to use a dataset of Wikipedia links from [DBpedia](http://wiki.dbpedia.org/).  DBpedia provides structured Wikipedia data available in 125 languages.  

"*The full DBpedia data set features 38 million labels and abstracts in 125 different languages, 25.2 million links to images and 29.8 million links to external web pages; 80.9 million links to Wikipedia categories, and 41.2 million links to [YAGO](http://www.mpi-inf.mpg.de/departments/databases-and-information-systems/research/yago-naga/yago/) categories*" --[about DBpedia](http://wiki.dbpedia.org/about)

Today's lesson is inspired by this [SciKit Learn Example](http://scikit-learn.org/stable/auto_examples/applications/wikipedia_principal_eigenvector.html#sphx-glr-auto-examples-applications-wikipedia-principal-eigenvector-py)

### Imports

In [4]:
import os, numpy as np, pickle
from bz2 import BZ2File
from datetime import datetime
from pprint import pprint
from time import time
from tqdm import tqdm_notebook
from scipy import sparse

from sklearn.decomposition import randomized_svd
from sklearn.externals.joblib import Memory
from urllib.request import urlopen

C:\Users\Dell\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


### Downloading the data

The data we have are:
- **redirects**: URLs that redirect to other URLs
- **links**: which pages link to which other pages

Note: this takes a while

In [6]:
PATH = 'data/dbpedia/'
URL_BASE = 'http://downloads.dbpedia.org/3.5.1/en/'
filenames = ["redirects_en.nt.bz2", "page_links_en.nt.bz2"]

for filename in filenames:
    if not os.path.exists(PATH):
      os.mkdir(PATH)
    if not os.path.exists(PATH+filename):
        print("Downloading '%s', please wait..." % filename)
        open(PATH+filename, 'wb').write(urlopen(URL_BASE+filename).read())

In [7]:
redirects_filename = PATH+filenames[0]
page_links_filename = PATH+filenames[1]

### Graph Adjacency Matrix

We will construct a graph **adjacency matrix**, of which pages point to which.

<img src="images/graph.png" alt="" style="width: 25%"/>
(source: [PageRank and HyperLink Induced Topic Search](https://www.slideshare.net/priyabrata232/page-rank-and-hyperlink))

<img src="images/adjaceny_matrix.png" alt="" style="width: 80%"/>
(source: [PageRank and HyperLink Induced Topic Search](https://www.slideshare.net/priyabrata232/page-rank-and-hyperlink))

The power $A^2$ will give you how many ways there are to get from one page to another in 2 steps.  You can see a more detailed example, as applied to airline travel, [worked out in these notes](http://www.utdallas.edu/~jwz120030/Teaching/PastCoursesUMBC/M221HS06/ProjectFiles/Adjacency.pdf).

We want to keep track of which pages point to which pages.  We will store this in a  square matrix, with a $1$ in position $(r, c)$ indicating that the topic in row $r$ points to the topic in column $c$

You can read [more about graphs here](http://www.geeksforgeeks.org/graph-and-its-representations/).

### Data Format

One line of the file looks like:
- `<http://dbpedia.org/resource/AfghanistanHistory> <http://dbpedia.org/property/redirect> <http://dbpedia.org/resource/History_of_Afghanistan> .`

In the below slice, the plus 1, -1 are to remove the <>

In [22]:
DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)

In [24]:
def get_lines(filename): return (line.split() for line in BZ2File(filename))

Loop through redirections and create dictionary of source to final destination

In [25]:
def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ

In [26]:
def get_redirects(redirects_filename):
    redirects={}
    lines = get_lines(redirects_filename)
    return {src[SLICE]:get_redirect(targ[SLICE], redirects) 
                for src,_,targ,_ in tqdm_notebook(lines, leave=False)}

In [29]:
redirects = get_redirects(redirects_filename)

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [31]:
mem_usage()

0.044150988550025305

In [32]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    lst.append(index_map.setdefault(redirects.get(k, k), len(index_map)))

In [33]:
limit=119077682 #5000000

In [34]:
# Computing the integer index map
index_map = dict() # links->IDs
lines = get_lines(page_links_filename)
source, destination, data = [],[],[]
for l, split in tqdm_notebook(enumerate(lines), total=limit):
    if l >= limit: break
    add_item(source, redirects, index_map, split[0])
    add_item(destination, redirects, index_map, split[2])
    data.append(1)

C:\Users\Dell\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


In [36]:
n=len(data); n

119077682

In [37]:
type(data)

list

### Looking at our data

The below steps are just to illustrate what info is in our data and how it is structured.  They are not efficient.

Let's see what type of items are in index_map:

In [39]:
index_map.popitem()

(b'1st_Bengal_Infantry_Regiment%282nd_in_Command%29', 9991176)

Let's look at one item in our index map:

1940_Cincinnati_Reds_Team_Issue has index $9991173$.  This only shows up once in the destination list:

In [40]:
[i for i,x in enumerate(source) if x == 9991173]

[119077649]

In [43]:
type(source)

list

In [428]:
source[119077649], destination[119077649]

(9991173, 9991050)

Now, we want to check which page is the source (has index $9991050$).  Note: usually you should not access a dictionary by searching for its values.  This is inefficient and not how dictionaries are intended to be used.

In [44]:
for page_name, index in index_map.items():
    if index == 9991050:
        print(page_name)

b'W711-2'


We can see on Wikipedia that the Cincinati Red Teams Issue has [redirected to W711-2](https://en.wikipedia.org/wiki/W711-2):

<img src="images/cincinnati_reds.png" alt="" style="width: 70%"/>

In [45]:
test_inds = [i for i,x in enumerate(source) if x == 9991050]

In [46]:
len(test_inds)

47

In [47]:
test_inds[:5]

[119076756, 119076757, 119076758, 119076759, 119076760]

In [48]:
test_dests = [destination[i] for i in test_inds]

Now, we want to check which page is the source (has index 9991174):

In [435]:
for page_name, index in index_map.items():
    if index in test_dests:
        print(page_name)

b'Baseball'
b'Ohio'
b'Cincinnati'
b'Flash_Thompson'
b'1940'
b'1938'
b'Lonny_Frey'
b'Cincinnati_Reds'
b'Ernie_Lombardi'
b'Baseball_card'
b'James_Wilson'
b'Trading_card'
b'Detroit_Tigers'
b'Baseball_culture'
b'Frank_McCormick'
b'Bucky_Walters'
b'1940_World_Series'
b'Billy_Werber'
b'Ival_Goodman'
b'Harry_Craft'
b'Paul_Derringer'
b'Johnny_Vander_Meer'
b'Cigarette_card'
b'Eddie_Joost'
b'Myron_McCormick'
b'Beckett_Media'
b'Icarus_affair'
b'Ephemera'
b'Sports_card'
b'James_Turner'
b'Jimmy_Ripple'
b'Lewis_Riggs'
b'The_American_Card_Catalog'
b'Rookie_card'
b'Willard_Hershberger'
b'Elmer_Riddle'
b'Joseph_Beggs'
b'Witt_Guise'
b'Milburn_Shoffner'


We can see that the items in the list appear in the wikipedia page:

<img src="images/cincinnati_reds2.png" alt="" style="width: 100%"/>
(Source: [Wikipedia](https://en.wikipedia.org/wiki/W711-2))


### Create Matrix

Below we create a sparse matrix using Scipy's COO format, and that convert it to CSR.

**Questions**: What are COO and CSR?  Why would we create it with COO and then convert it right away?

In [49]:
X = sparse.coo_matrix((data, (destination,source)), shape=(n,n), dtype=np.float32)
X = X.tocsr()

In [50]:
del(data,destination, source)

In [52]:
X

<119077682x119077682 sparse matrix of type '<class 'numpy.float32'>'
	with 93985520 stored elements in Compressed Sparse Row format>

In [53]:
names = {i: name for name, i in index_map.items()}

In [54]:
mem_usage()

0.2057205484369489

### Save matrix so we don't have to recompute

In [55]:
pickle.dump(X, open(PATH+'X.pkl', 'wb'))
pickle.dump(index_map, open(PATH+'index_map.pkl', 'wb'))

In [8]:
X = pickle.load(open(PATH+'X.pkl', 'rb'))
index_map = pickle.load(open(PATH+'index_map.pkl', 'rb'))

In [9]:
names = {i: name for name, i in index_map.items()}

In [10]:
X.shape


(119077682, 119077682)

## Power method

### Motivation

An $n \times n$ matrix $A$ is **diagonalizable** if it has $n$ linearly independent eigenvectors $v_1,\, \ldots v_n$.

Then any $w$ can be expressed $w = \sum_{j=1}^n c_j v_j $, for some scalars $c_j$.

**Exercise:** Show that $$ A^k w = \sum_{j=1}^n c_j \lambda_j^k v_j$$

**Question**: How will this behave for large $k$?

This is inspiration for the **power method**.

### Code

In [11]:
def show_ex(v):
    print(', '.join(names[i].decode() for i in np.abs(v.squeeze()).argsort()[-1:-10:-1]))

In [12]:
?np.squeeze

How to normalize a sparse matrix:

In [19]:
S = sparse.csr_matrix(np.array([[1,2],[3,4]]))
Sr = S.sum(axis=0).A1; Sr

array([4, 6], dtype=int32)

[numpy.matrix.A1](https://docs.scipy.org/doc/numpy/reference/generated/numpy.matrix.A1.html#numpy.matrix.A1)

In [14]:
S.indices


array([0, 1, 0, 1], dtype=int32)

In [15]:
S.data

array([1, 2, 3, 4], dtype=int32)

In [16]:
S.data / np.take(Sr, S.indices)

array([0.25      , 0.33333333, 0.75      , 0.66666667])

In [25]:
def power_method(A, max_iter=100):
    n = A.shape[1]
    print(type(A))
#     A = np.copy(A)
    print(type(A))
    A.data /= np.take(A.sum(axis=0).A1, A.indices)

    scores = np.ones(n, dtype=np.float32) * np.sqrt(A.sum()/(n*n)) # initial guess
    for i in range(max_iter):
        scores = A @ scores
        nrm = np.linalg.norm(scores)
        scores /= nrm
        print(nrm)

    return scores

**Question**: Why normalize the scores on each iteration?

In [26]:
scores = power_method(X, max_iter=10)

<class 'scipy.sparse.csr.csr_matrix'>
<class 'scipy.sparse.csr.csr_matrix'>
0.62119865
0.8561402
1.02793
1.0202881
1.0264537
1.0250401
1.0236456
1.0212563
1.0190037
1.0167882


In [29]:
scores

array([6.6939392e-05, 8.6461741e-06, 3.6246238e-05, ..., 0.0000000e+00,
       0.0000000e+00, 0.0000000e+00], dtype=float32)

In [346]:
show_ex(scores)

Living_people, Year_of_birth_missing_%28living_people%29, United_States, United_Kingdom, Race_and_ethnicity_in_the_United_States_Census, France, Year_of_birth_missing, World_War_II, Germany


In [30]:
mem_usage()

0.16398659449256747

### Comments

Many advanced eigenvalue algorithms that are used in practice are variations on the power method.

In Lesson 3: Background Removal, we used Facebook's [fast randomized pca/svd library, fbpca](https://github.com/facebook/fbpca).  Check out [the source code](https://github.com/facebook/fbpca/blob/master/fbpca.py#L1549) for the pca method we used. It uses the power method!

**Further Study**

- Check out [Google Page Rank, Power Iteration and the Second EigenValue of the Google Matrix](http://rstudio-pubs-static.s3.amazonaws.com/239261_8a607707294341c4b7e26acf728c28bd.html) for animations of the distribution as it converges.

- The convergence rate of the power method is the ratio of the largest eigenvalue to the 2nd largest eigenvalue.  It can be speeded up by adding *shifts*.  To find eigenvalues other than the largest, a method called *deflation* can be used.  See Chapter 12.1 of Greenbaum & Chartier for more details.

- *Krylov Subspaces*: In the Power Iteration, notice that we multiply by our matrix A each time, effectively calculating $$Ab,\,A^2b,\,A^3b,\,A^4b\, \ldots$$
  
  The matrix with those vectors as columns is called the *Krylov matrix*, and the space spanned by those vectors is the *Krylov subspace*.  Keep this in mind for later.

### Compare to SVD

In [27]:
%time U, s, V = randomized_svd(X, 3, n_iter=3)

CPU times: user 8min 40s, sys: 1min 20s, total: 10min 1s
Wall time: 5min 56s


In [28]:
mem_usage()

28.353073152

In [30]:
# Top wikipedia pages according to principal singular vectors
show_ex(U.T[0])

List_of_World_War_II_air_aces, List_of_animated_feature-length_films, List_of_animated_feature_films:2000s, International_Swimming_Hall_of_Fame, List_of_museum_ships, List_of_linguists, List_of_television_programs_by_episode_count, List_of_game_show_hosts, List_of_astronomers


In [31]:
show_ex(U.T[1])

List_of_former_United_States_senators, List_of_United_States_Representatives_from_New_York, List_of_United_States_Representatives_from_Pennsylvania, Members_of_the_110th_United_States_Congress, List_of_Justices_of_the_Ohio_Supreme_Court, Congressional_endorsements_for_the_2008_United_States_presidential_election, List_of_United_States_Representatives_in_the_110th_Congress_by_seniority, List_of_Members_of_the_United_States_House_of_Representatives_in_the_103rd_Congress_by_seniority, List_of_United_States_Representatives_in_the_111th_Congress_by_seniority


In [32]:
show_ex(V[0])

United_States, Japan, United_Kingdom, Germany, Race_and_ethnicity_in_the_United_States_Census, France, United_States_Army_Air_Forces, Australia, Canada


In [33]:
show_ex(V[1])

Democratic_Party_%28United_States%29, Republican_Party_%28United_States%29, Democratic-Republican_Party, United_States, Whig_Party_%28United_States%29, Federalist_Party, National_Republican_Party, Catholic_Church, Jacksonian_democracy


**Exercise:** Normalize the data in various ways.  Don't overwrite the adjacency matrix, but instead create a new one.  See how your results differ.

**Eigen Decomposition vs SVD:**
- SVD involves 2 bases, eigen decomposition involves 1 basis
- SVD bases are orthonormal, eigen basis generally not orthogonal
- All matrices have an SVD, not all matrices (not even all square) have an eigen decomposition

## QR Algorithm

We used the power method to find the eigenvector corresponding to the largest eigenvalue of our matrix of Wikipedia links.  This eigenvector gave us the relative importance of each Wikipedia page (like a simplified PageRank).

Next, let's look at a method for finding all eigenvalues of a symmetric, positive definite matrix.  This method includes 2 fundamental algorithms in numerical linear algebra, and is a basis for many more complex methods.

[The Second Eigenvalue of the Google Matrix](https://nlp.stanford.edu/pubs/secondeigenvalue.pdf): has "implications for the convergence rate of the standard PageRank algorithm as the web scales, for the stability of PageRank to perturbations to the link structure of the web, for the detection of Google spammers, and for the design of algorithms to speed up PageRank".


### Avoiding Confusion: QR Algorithm vs QR Decomposition

The **QR algorithm** uses something called the **QR decomposition**.  Both are important, so don't get them confused.  The **QR decomposition** decomposes a matrix $A = QR$ into a set of orthonormal columns $Q$ and a triangular matrix $R$.  We will look at several ways to calculate the QR decomposition in a future lesson.  For now, just know that it is giving us an orthogonal matrix and a triangular matrix.

### Linear Algebra

Two matrices $A$ and $B$ are **similar** if there exists a non-singular matrix $X$ such that $$B = X^{-1}AX$$

Watch this: [Change of Basis](https://www.youtube.com/watch?v=P2LTAUO1TdA&index=13&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)

**Theorem**: If $X$ is non-singular, then $A$ and $X^{-1}AX$ have the same eigenvalues.

#### More Linear Algebra

A **Schur factorization** of a matrix $A$ is a factorization:
$$ A = Q T Q^*$$
where $Q$ is unitary and $T$ is upper-triangular.

**Question**: What can you say about the eigenvalues of $A$?

**Theorem:** Every square matrix has a Schur factorization.

#### Other resources

Review: [Linear combinations, span, and basis vectors](https://www.youtube.com/watch?v=k7RM-ot2NWY&index=3&list=PLZHQObOWTQDPD3MizzM2xVFitgF8hE_ab)

See Lecture 24 for proofs of above theorems (and more!)

### Algorithm

The most basic version of the QR algorithm:

    for k=1,2,...
        Q, R = A
        A = R @ Q
        
Under suitable assumptions, this algorithm converges to the Schur form of A!

#### Why it works

Written again, only with subscripts:

$A_0 = A$

for $k=1,2,\ldots$

   $\quad Q_k$, $R_k$ = $A_{k-1}$
    
   $\quad A_k$ = $R_k Q_k$
        
We can think of this as constructing sequences of $A_k$, $Q_k$, and $R_k$.

$$ A_k = Q_k \, R_k $$

$$ Q_k^{-1} \, A_k = R_k$$

Thus, 

$$ R_k Q_k = Q_k^{-1} \, A_k \, Q_k $$

$$A_k = Q_k^{-1} Q_2^{-1} Q_1^{-1} A Q_1 Q_2 \dots Q_k$$

Trefethen proves the following on page 216-217:

$$A^k = Q_1 Q_2 \dots Q_k R_k R_{k-1}\dots R_1$$

**Key**: The QR algorithm constructs orthonormal bases for successive powers $A^k$.  And remember the close relationship between powers of A and the eigen decomposition.

To learn more, read up on *Rayleigh quotients*.

#### Pure QR

In [36]:
import numpy as np
n = 6
A = np.random.rand(n,n)
AT = A @ A.T

In [37]:
def pure_qr(A, max_iter=50000):
    Ak = np.copy(A)
    n = A.shape[0]
    QQ = np.eye(n)
    for k in range(max_iter):
        Q, R = np.linalg.qr(Ak)
        Ak = R @ Q
        QQ = QQ @ Q
        if k % 100 == 0:
            print(Ak)
            print("\n")
    return Ak, QQ

#### Pure QR

In [38]:
Ak, Q = pure_qr(A)

[[ 2.89008764 -0.99996943  0.86017026 -0.91863518  0.30377902 -0.34002983]
 [-1.28155424  0.86258255  0.38097686 -0.0324908  -0.59940268 -0.06352591]
 [ 0.54821388 -0.20867779  0.06160238 -0.36892357  0.37038234  0.3706878 ]
 [-0.11459493  0.42684517 -0.47282993  0.01609928 -0.41491812  0.09428299]
 [ 0.1055986  -0.21634265 -0.10009137  0.20657669 -0.06058954  0.24311371]
 [ 0.05260529 -0.11723463 -0.10461014 -0.25680982  0.08327512 -0.04295644]]


[[ 3.61788870e+000 -1.60298098e-001  2.36677528e-001 -5.07902767e-001
  -4.05196852e-001 -2.90695358e-001]
 [-4.18315959e-076  5.59666029e-001 -3.73166441e-001  3.00707870e-001
  -1.72811302e-002 -1.58940736e-001]
 [-4.29431522e-076  6.54333805e-001  2.92538857e-001 -9.07859459e-002
   3.21217543e-001 -4.37013417e-001]
 [ 3.45866927e-097 -2.94532683e-021 -2.11323000e-021 -2.22156741e-001
  -3.24441201e-001  9.89856026e-002]
 [ 4.42174646e-097 -2.17405732e-021  7.36118413e-022  3.78275706e-001
  -1.67360678e-001 -3.89650142e-001]
 [ 1.3730472

[[ 3.61788870e+000  7.52939764e-002 -2.75758136e-001  3.61945661e-002
   6.48723728e-001 -2.90690521e-001]
 [ 0.00000000e+000  5.44698745e-001  6.67170854e-001  8.89268580e-002
  -2.49544321e-001 -4.35194687e-001]
 [ 0.00000000e+000 -3.60329393e-001  3.07506141e-001 -3.57177974e-001
  -6.62420508e-002  1.63861703e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -2.18429951e-001
  -3.81603420e-001 -3.56533721e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.21111420e-001
  -1.71090161e-001  1.85773006e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  1.39600878e-192
   1.61266083e-193 -3.53747606e-001]]


[[ 3.61788870e+000  1.42346198e-001  2.47889677e-001  3.89671107e-001
   5.19912445e-001 -2.90690521e-001]
 [ 0.00000000e+000  5.78950909e-001  3.94417186e-001  2.82099367e-001
   8.00535502e-002  1.90842010e-001]
 [ 0.00000000e+000 -6.33083060e-001  2.73253978e-001  1.90048772e-001
  -2.82908658e-001 -4.24057073e-001]
 [ 0.00000000e+000  0.00000000e+00

[[ 3.61788870e+000  2.85801328e-001  5.41601719e-003 -5.58195284e-001
  -3.32521499e-001 -2.90690521e-001]
 [ 0.00000000e+000  2.48846231e-001  4.35116551e-001 -2.08240927e-001
   3.03525047e-001 -2.65520440e-001]
 [ 0.00000000e+000 -5.92383695e-001  6.03358655e-001  2.14829990e-001
   1.43181026e-001 -3.81764285e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -2.13908904e-001
  -3.18064687e-001  4.51911631e-002]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.84650153e-001
  -1.75611208e-001 -3.99481742e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000 -2.15524615e-001  1.87778786e-001 -6.44124766e-001
   8.51810315e-002 -2.90690521e-001]
 [ 0.00000000e+000  3.30253206e-001  6.82320367e-001 -1.91071299e-001
  -2.58285421e-001  4.53046545e-001]
 [ 0.00000000e+000 -3.45179880e-001  5.21951681e-001 -1.74139455e-001
   2.61916625e-001  1.04851810e-001]
 [ 0.00000000e+000  0.00000000e+00

[[ 3.61788870e+000  2.84526753e-001  2.75001661e-002 -3.85829187e-002
  -6.48586063e-001 -2.90690521e-001]
 [ 0.00000000e+000  2.63090762e-001  4.08725388e-001 -3.67161549e-001
  -2.16671929e-002 -2.35204437e-001]
 [ 0.00000000e+000 -6.18774859e-001  5.89114125e-001 -2.61987226e-002
   2.57256731e-001 -4.01153271e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -2.18652039e-001
  -3.81428296e-001  3.55847289e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.21286544e-001
  -1.70868073e-001 -1.87084502e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000 -2.23698028e-001  1.77963267e-001 -3.91823367e-001
  -5.18292361e-001 -2.90690521e-001]
 [ 0.00000000e+000  3.15589844e-001  6.73092424e-001  1.44312257e-001
  -2.92323942e-001  4.47910250e-001]
 [ 0.00000000e+000 -3.54407822e-001  5.36615042e-001 -3.07593045e-001
  -3.53789828e-002  1.24985925e-001]
 [ 0.00000000e+000  0.00000000e+00

   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000  2.81384469e-001  5.03439474e-002  4.74960450e-001
  -4.43356621e-001 -2.90690521e-001]
 [ 0.00000000e+000  2.82083999e-001  3.83897549e-001 -1.99444508e-001
  -3.07253257e-001 -2.02105663e-001]
 [ 0.00000000e+000 -6.43602697e-001  5.70120888e-001 -2.31687069e-001
   1.19511067e-001 -4.18805892e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -1.60760210e-001
  -3.69220648e-001  3.71043070e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.33494192e-001
  -2.28759902e-001  1.54773847e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000 -2.31639416e-001  1.67496009e-001  1.71104365e-001
  -6.26798068e-001 -2.90690521e-001]
 [ 0.00000000e+000  3.01427968e-001  6.62273786e-001  3.26616547e-001
  -5.20067725e-002  4.41693813e-001]
 [ 0.00000000e+000 -3.65226460e-001  5.50776918e-001 -1.44987062e-001
  -2.67843512e-001  1.45436066e

...

[[ 3.61788870e+000 -2.66568322e-001  1.03213672e-001 -6.01587241e-001
  -2.45449209e-001 -2.90690521e-001]
 [ 0.00000000e+000  2.44563533e-001  5.81916298e-001  6.00930544e-002
  -3.48014277e-001  3.90312158e-001]
 [ 0.00000000e+000 -4.45583948e-001  6.07641353e-001 -2.77199522e-001
   2.40786407e-002  2.52787446e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -2.03224592e-001
  -3.13894958e-001 -1.49625174e-002]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.88819882e-001
  -1.86295519e-001 -4.01751200e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000  1.62970164e-001 -2.34845604e-001 -6.25134937e-001
   1.77084253e-001 -2.90690521e-001]
 [ 0.00000000e+000  4.21809131e-001  7.07617528e-001  2.52358601e-001
   1.51496757e-001 -4.64928563e-001]
 [ 0.00000000e+000 -3.19882718e-001  4.30395756e-001  7.50709091e-002
  -3.31470506e-001  9.30080029e-003]
 [ 0.00000000e+000  0.00000000e+00

[[ 3.61788870e+000  1.67212945e-002 -2.85363156e-001 -2.63183939e-001
  -5.94042706e-001 -2.90690521e-001]
 [ 0.00000000e+000  5.96585145e-001  6.06154949e-001  7.87589674e-002
   2.45879106e-001 -3.91972248e-001]
 [ 0.00000000e+000 -4.21345297e-001  2.55619742e-001  3.48786616e-001
  -1.17613159e-001  2.50205577e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -2.32517581e-001
  -3.58389754e-001  2.67841955e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.44325086e-001
  -1.57002530e-001 -2.99814261e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000  2.20869414e-001  1.81461936e-001 -5.63977893e-001
  -3.22616581e-001 -2.90690521e-001]
 [ 0.00000000e+000  4.61650994e-001  3.23121416e-001 -2.93901062e-001
   1.54564651e-001  2.83990233e-002]
 [ 0.00000000e+000 -7.04378830e-001  3.90553892e-001  4.47243887e-002
   2.99793482e-001 -4.64153605e-001]
 [ 0.00000000e+000  0.00000000e+00

[[ 3.61788870e+000  2.74649344e-001 -7.92431064e-002  1.63596470e-001
  -6.28799425e-001 -2.90690521e-001]
 [ 0.00000000e+000  2.35395755e-001  5.48877926e-001 -3.56882080e-001
  -3.74191382e-002 -3.66435769e-001]
 [ 0.00000000e+000 -4.78622320e-001  6.16809131e-001  2.91492707e-002
   2.69313531e-001 -2.86303861e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -1.96430104e-001
  -3.89727922e-001  3.96228776e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.12986919e-001
  -1.93090007e-001 -6.80489563e-002]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000 -1.77088327e-001  2.24391303e-001 -1.92609229e-001
  -6.20527362e-001 -2.90690521e-001]
 [ 0.00000000e+000  3.98069022e-001  7.05628029e-001  1.65356072e-001
  -2.51469849e-001  4.64621898e-001]
 [ 0.00000000e+000 -3.21872218e-001  4.54135864e-001 -3.23692410e-001
  -8.23891780e-002  1.92760503e-002]
 [ 0.00000000e+000  0.00000000e+00

[[ 3.61788870e+000 -4.18341613e-002  2.82774884e-001  4.01092241e-001
   5.11153144e-001 -2.90690521e-001]
 [ 0.00000000e+000  5.77688307e-001  6.34682870e-001  1.08951886e-001
   2.35752857e-001  4.12519614e-001]
 [ 0.00000000e+000 -3.92817376e-001  2.74516579e-001  3.20050861e-001
  -1.79616235e-001 -2.14645386e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -2.31307701e-001
  -3.39552588e-001 -1.85953426e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.63162252e-001
  -1.58212410e-001  3.56439655e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000 -1.91009000e-001 -2.12667097e-001  6.30019732e-001
   1.58832172e-001 -2.90690521e-001]
 [ 0.00000000e+000  5.16822243e-001  3.42364822e-001 -2.57320103e-001
   1.83709958e-001 -9.80048372e-002]
 [ 0.00000000e+000 -6.85135424e-001  3.35382644e-001  8.60142109e-002
   3.07868543e-001  4.54576864e-001]
 [ 0.00000000e+000  0.00000000e+00

[[ 3.61788870e+000 -5.57095639e-002  2.80371498e-001 -1.61339327e-001
   6.29382351e-001 -2.90690521e-001]
 [ 0.00000000e+000  5.65056001e-001  6.49008809e-001 -1.37170489e-001
   2.22559277e-001  4.22589958e-001]
 [ 0.00000000e+000 -3.78491437e-001  2.87148885e-001  3.30983711e-001
   1.55710877e-001 -1.94069064e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000 -1.96705401e-001
  -3.89714950e-001 -3.95982071e-001]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  3.12999891e-001
  -1.92814711e-001  6.94701605e-002]
 [ 0.00000000e+000  0.00000000e+000  0.00000000e+000  4.94065646e-324
   0.00000000e+000 -3.53747606e-001]]


[[ 3.61788870e+000 -1.71934146e-001 -2.28364581e-001  1.94997591e-001
   6.19780979e-001 -2.90690521e-001]
 [ 0.00000000e+000  5.44953844e-001  3.60526927e-001 -2.71502742e-001
  -1.42630452e-001 -1.36886827e-001]
 [ 0.00000000e+000 -6.66973319e-001  3.07251042e-001 -2.41432969e-001
   2.23153322e-001  4.44417676e-001]
 [ 0.00000000e+000  0.00000000e+00

In [42]:
sorted(np.diag(Ak))

[-0.35374760588401694,
 -0.23196048244859024,
 -0.15755962902649528,
 0.27783928676591724,
 0.5743655994663476,
 3.6178887002578657]

Let's compare to the eigenvalues:

In [41]:
sorted(np.linalg.eigvals(A))

[(-0.35374760588401655+0j),
 (-0.1947600557375426-0.3492519896006308j),
 (-0.1947600557375426+0.3492519896006308j),
 (0.4261024431161381-0.4757480275180824j),
 (0.4261024431161381+0.4757480275180824j),
 (3.617888700257862+0j)]

Check that Q is orthogonal:

In [43]:
np.allclose(np.eye(n), Q @ Q.T), np.allclose(np.eye(n), Q.T @ Q)

(True, True)

This is really really slow.

#### Practical QR (QR with shifts)

**Idea**: Instead of factoring $A_k$ as $Q_k R_k$, 

1. Get the QR factorization $$A_k - s_k I = Q_k R_k$$
2. Set $$A_{k+1} = R_k Q_k + s_k I$$

Choose $s_k$ to approximate an eigenvalue of $A$.  We'll use $s_k = A_k(m,m)$. 

The idea of adding shifts to speed up convergence shows up in many algorithms in numerical linear algebra (including the power method, inverse iteration, and Rayleigh quotient iteration).   

#### Homework: Add shifts to the QR algorithm

In [44]:
#Exercise: Add shifts to the QR algorithm
#Exercise: def practical_qr(A, iters=10):
#Exercise:     return Ak, Q


def practical_qr(A, max_iter=10):
    Ak = np.copy(A)
    n = A.shape[0]
    QQ = np.eye(n)
    for k in range(max_iter):
        shift = Ak[n-1,n-1] * np.eye(n)
        Q, R = np.linalg.qr(Ak - shift)
        Ak = R @ Q + shift
        QQ = QQ @ Q
        if k % 100 == 0:
            print(Ak)
            print("\n")
    return Ak, QQ


In [45]:
Ak, Q = practical_qr(A, 50000)

[[ 2.44457325 -1.39728933 -0.7907897  -0.80637926 -0.52623786  0.18010722]
 [-1.69971287  0.94311346 -0.14940658 -0.09478158  0.71214388  0.23242719]
 [-0.58890007  0.16595329 -0.0494239   0.21440176  0.07524361 -0.27505997]
 [-0.0392854   0.24069347  0.51337944  0.06857008  0.46399021 -0.30395747]
 [-0.15956376  0.28824941 -0.19149625 -0.48127393  0.03915403 -0.0484185 ]
 [ 0.08055898 -0.15839877  0.18415151 -0.25107937 -0.25034733  0.28083895]]


[[ 3.61788870e+000  2.59478219e-001 -1.19928253e-001 -6.24401723e-001
  -1.79652471e-001 -2.90690521e-001]
 [ 1.34954401e-065  2.54664249e-001  6.04369927e-001 -5.12210871e-003
   3.48332726e-001 -4.05572761e-001]
 [-1.71793491e-065 -4.23130320e-001  5.97540638e-001  2.73907864e-001
  -7.58750746e-002 -2.27499032e-001]
 [ 5.94421029e-102  7.89223540e-037 -1.37981942e-036 -1.95057642e-001
  -3.12951745e-001 -5.78753274e-002]
 [ 7.17318981e-102  5.70100520e-037 -1.12435230e-037  3.89763095e-001
  -1.94462470e-001 -3.97842117e-001]
 [ 0.0000000

 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887   0.24555322 -0.14633983  0.48267286  0.43494762 -0.29069052]
 [ 0.          0.27719655  0.63796782  0.12713764 -0.3146136  -0.42709044]
 [ 0.         -0.38953242  0.57500834 -0.29434143 -0.01902361 -0.18395333]
 [ 0.          0.          0.         -0.22518379 -0.32791642 -0.12219193]
 [ 0.          0.          0.          0.37479842 -0.16433632  0.38301049]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.11231079 -0.26286502  0.52655596 -0.38064595 -0.29069052]
 [ 0.          0.60253184  0.43327856 -0.11709157  0.25589858 -0.23922891]
 [ 0.         -0.59422169  0.24967305  0.26770998  0.22645378  0.39876635]
 [ 0.          0.          0.         -0.15739654 -0.36024853  0.34883806]
 [ 0.          0.          0.          0.34246631 -0.23212357  0.19984972]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0

[[ 3.6178887   0.02629952  0.28464024  0.44279098 -0.47548782 -0.29069052]
 [ 0.          0.61632842  0.55139428  0.1561158  -0.20834581  0.34997743]
 [ 0.         -0.47610596  0.23587647 -0.27174668 -0.24600425 -0.30620397]
 [ 0.          0.          0.         -0.16358524 -0.37378988  0.38095881]
 [ 0.          0.          0.          0.32892496 -0.22593487  0.12844567]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887   0.2302615  -0.16938528 -0.26097441 -0.59501671 -0.29069052]
 [ 0.          0.3038745   0.66429358 -0.21900314  0.24672402 -0.44286874]
 [ 0.         -0.36320666  0.54833039  0.28293627  0.11513651 -0.14181803]
 [ 0.          0.          0.         -0.23246427 -0.3586702   0.26895434]
 [ 0.          0.          0.          0.34404464 -0.15705584 -0.29881678]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.15271019 -0.24164298 -0.63628751  0.13149423 -0.29069052]
 [ 0.          0.56

[[ 3.6178887  -0.19177204 -0.21197928  0.62976891  0.15982379 -0.29069052]
 [ 0.          0.51558807  0.3417172  -0.25729996  0.18441035 -0.09637056]
 [ 0.         -0.68578304  0.33661681  0.08645467  0.30734283  0.45492613]
 [ 0.          0.          0.         -0.19263006 -0.3130097   0.07042318]
 [ 0.          0.          0.          0.38970514 -0.19689005  0.39581369]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.15438425  0.24057688  0.3288871  -0.56034436 -0.29069052]
 [ 0.          0.43581126  0.70742186  0.28051994 -0.07566714  0.46428956]
 [ 0.         -0.32007839  0.41639362 -0.18535787 -0.28873859 -0.02608206]
 [ 0.          0.          0.         -0.17604552 -0.38489621  0.39976698]
 [ 0.          0.          0.          0.31781863 -0.2134746   0.04259421]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887   0.27696125  0.07074036 -0.39469841 -0.51610628 -0.29069052]
 [ 0.          0.30

[[ 3.61788870e+00 -2.26585191e-01 -1.74272440e-01 -4.95703414e-01
  -4.20036484e-01 -2.90690521e-01]
 [ 0.00000000e+00  4.49335357e-01  3.21231981e-01  3.17335400e-01
  -1.08069381e-01 -1.34725968e-02]
 [ 0.00000000e+00 -7.06268265e-01  4.02869529e-01  4.90538080e-04
  -2.99605542e-01  4.64826379e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.23699253e-01
  -3.26106533e-01  1.10463194e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  3.76608308e-01
  -1.65820859e-01 -3.86556317e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00 -3.53747606e-01]]


[[ 3.6178887  -0.13393892  0.25253138 -0.51581201  0.3950829  -0.29069052]
 [ 0.          0.4676332   0.70316554 -0.28215831 -0.0075852   0.46053685]
 [ 0.         -0.32433471  0.38457169  0.08653011  0.3390961  -0.06442735]
 [ 0.          0.          0.         -0.15794614 -0.36230366 -0.35423909]
 [ 0.          0.          0.          0.34041118 -0.23157397 -0.190112  ]
 [ 0.          0

...

 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.24053657 -0.15444704 -0.41948675  0.49616871 -0.29069052]
 [ 0.          0.41649473  0.32007334 -0.13341697 -0.3160881   0.02596087]
 [ 0.         -0.7074269   0.43571015 -0.28934191  0.02671999  0.46429636]
 [ 0.          0.          0.         -0.16587691 -0.3766724  -0.38667853]
 [ 0.          0.          0.          0.32604244 -0.2236432  -0.11003464]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.12396073  0.25757614  0.29077498  0.58103565 -0.29069052]
 [ 0.          0.48230976  0.69934037 -0.04692605  0.27463628  0.457665  ]
 [ 0.         -0.32815987  0.36989513  0.35143189 -0.03182696 -0.08238826]
 [ 0.          0.          0.         -0.23301062 -0.35481842 -0.25347516]
 [ 0.          0.          0.          0.34789642 -0.15650949  0.31205488]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887   0

[[ 3.6178887  -0.26437542 -0.10870774  0.57961602 -0.2935946  -0.29069052]
 [ 0.          0.34863996  0.33597901  0.04952767  0.353336    0.10897353]
 [ 0.         -0.69152123  0.50356493  0.27243372  0.02500034  0.45207283]
 [ 0.          0.          0.         -0.15647766 -0.34826827  0.31328274]
 [ 0.          0.          0.          0.35444657 -0.23304245  0.251956  ]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.10138047  0.2672709  -0.04579922 -0.64811647 -0.29069052]
 [ 0.          0.513242    0.6869832   0.11915665 -0.24383713  0.4488979 ]
 [ 0.         -0.34051704  0.33896288 -0.34830188 -0.08470785 -0.12139091]
 [ 0.          0.          0.         -0.21931545 -0.38088905  0.35374302]
 [ 0.          0.          0.          0.32182579 -0.17020466 -0.19103346]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887   0.28336212 -0.03765155 -0.63785666 -0.12365841 -0.29069052]
 [ 0.          0.23

[[ 3.6178887  -0.27918637 -0.06137346 -0.64937325  0.02160795 -0.29069052]
 [ 0.          0.29273748  0.372978    0.06174992 -0.35989062  0.18547518]
 [ 0.         -0.65452224  0.55946741 -0.25089456 -0.0765642   0.42643174]
 [ 0.          0.          0.         -0.17247304 -0.32007846 -0.17771442]
 [ 0.          0.          0.          0.38263638 -0.21704707 -0.3606182 ]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.07696467  0.27529651 -0.19302041  0.62039958 -0.29069052]
 [ 0.          0.5428295   0.66859778 -0.17248633  0.20153365  0.43618028]
 [ 0.         -0.35890246  0.30937539  0.31039995  0.18822591 -0.16121985]
 [ 0.          0.          0.         -0.19281356 -0.38971489 -0.39899327]
 [ 0.          0.          0.          0.31299995 -0.19670655  0.0493181 ]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887   0.27598452 -0.07445991  0.51819439  0.39195292 -0.29069052]
 [ 0.          0.23

[[ 3.6178887  -0.05572609 -0.28036821 -0.62294767  0.18463132 -0.29069052]
 [ 0.          0.62000157  0.51127371  0.22767645 -0.13596584 -0.316275  ]
 [ 0.         -0.51622653  0.23220332 -0.13347674 -0.33764649  0.3409035 ]
 [ 0.          0.          0.         -0.16004369 -0.33493005 -0.26289328]
 [ 0.          0.          0.          0.36778479 -0.22947642 -0.30416283]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.21702826  0.18603888 -0.05761192  0.64717338 -0.29069052]
 [ 0.          0.32755351  0.68075644 -0.29553276  0.12819041  0.45218838]
 [ 0.         -0.3467438   0.52465138  0.21517024  0.22819693  0.10849304]
 [ 0.          0.          0.         -0.20882681 -0.38709551 -0.3795718 ]
 [ 0.          0.          0.          0.31561933 -0.1806933   0.13248832]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887   0.18477052  0.21810911  0.6018317   0.24484919 -0.29069052]
 [ 0.          0.52

[[ 3.6178887   0.10757927 -0.26483661  0.30539583 -0.57348575 -0.29069052]
 [ 0.          0.50507837  0.69085407 -0.23622585  0.13730629 -0.45160398]
 [ 0.         -0.33664618  0.34712652  0.24470004  0.26002055  0.11090048]
 [ 0.          0.          0.         -0.17888564 -0.38633009  0.40118814]
 [ 0.          0.          0.          0.31638475 -0.21063447  0.02599962]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.28457137  0.02703452 -0.41878836 -0.49675832 -0.29069052]
 [ 0.          0.23568457  0.47708899  0.27641181 -0.24065496  0.30710678]
 [ 0.         -0.55041125  0.61652031 -0.18877425 -0.17941698  0.34918548]
 [ 0.          0.          0.         -0.23038935 -0.33701735  0.1733265 ]
 [ 0.          0.          0.          0.36569749 -0.15913076 -0.36274761]
 [ 0.          0.          0.          0.          0.         -0.35374761]]


[[ 3.6178887  -0.02120203  0.28506527 -0.57096323  0.31008631 -0.29069052]
 [ 0.          0.59

#### Practical QR

In [204]:
Ak, Q = practical_qr(A, 10)

[ 5.16264  2.53603  0.31923  0.35315  0.97569  0.43615]
[ 7.99381  0.05922  0.34478  0.29482  0.79026  0.29999]
[ 8.00491  0.04358  0.89735  0.26386  0.26182  0.31135]
[ 8.00493  0.13648  0.91881  0.14839  0.24313  0.33115]
[ 8.00493  0.43377  0.62809  0.13429  0.24592  0.33589]
[ 8.00493  0.81058  0.25128  0.13297  0.24722  0.3359 ]
[ 8.00493  0.98945  0.07221  0.13292  0.24747  0.3359 ]
[ 8.00493  1.0366   0.02497  0.13296  0.24751  0.3359 ]
[ 8.00493  1.04688  0.01465  0.13299  0.24752  0.3359 ]
[ 8.00493  1.04902  0.0125   0.13301  0.24753  0.3359 ]


Check that Q is orthogonal:

In [46]:
np.allclose(np.eye(n), Q @ Q.T), np.allclose(np.eye(n), Q.T @ Q)

(True, True)

Let's compare to the eigenvalues:

In [54]:
sorted(np.linalg.eigvals(A))

[(-0.35374760588401655+0j),
 (-0.1947600557375426-0.3492519896006308j),
 (-0.1947600557375426+0.3492519896006308j),
 (0.4261024431161381-0.4757480275180824j),
 (0.4261024431161381+0.4757480275180824j),
 (3.617888700257862+0j)]

In [53]:
sorted(np.diag(Ak))

[-0.35374760588401616,
 -0.22886857428483567,
 -0.16065153719024203,
 0.2522315598003208,
 0.5999733264298837,
 3.6178887002578635]

**Problem**: This is better than the unshifted version (which wasn't even guaranteed to converge), but is still really slow!  In fact, it is $\mathcal{O}(n^4)$, which is awful.

In the case of symmetric matrices, it's $\mathcal{O}(n^3)$

However, if you start with a **Hessenberg matrix** (zeros below the first subdiagonal), it's faster: $\mathcal{O}(n^3)$, and $\mathcal{O}(n^2)$ if symmetric.

## A Two-Phase Approach

In practice, a two phase approach is used to find eigenvalues:

1. Reduce the matrix to *Hessenberg* form (zeros below the first subdiagonal)
2. Iterative process that causes Hessenberg to converge to a *triangular* matrix.  The eigenvalues of a triangular matrix are the values on the diagonal, so we are finished!

<img src="images/nonhermitian_eigen.JPG" alt="2 phase approach" style="width: 80%"/>
(source: Trefethen, Lecture 25)

In the case of a Hermitian matrix, this approach is even faster, since the intermediate step is also Hermitian (and a Hermitian Hessenberg is *tridiagonal*).

<img src="images/hermitian_eigen.JPG" alt="2 phase approach" style="width: 80%"/>
(source: Trefethen, Lecture 25)

Phase 1 reaches an exact solution in a finite number of steps, whereas Phase 2 theoretically never reaches the exact solution.

We've already done step 2: the QR algorithm.  Remember that it would be possible to just use the QR algorithm, but ridiculously slow.

## Arnoldi Iteration

We can use the Arnoldi iteration for phase 1 (and the QR algorithm for phase 2).

#### Initializations

In [55]:
import numpy as np
n = 5
A0 = np.random.rand(n,n)  #.astype(np.float64)
A = A0 @ A0.T

np.set_printoptions(precision=5, suppress=True)

### Linear Algebra Review: Projections

When vector $\mathbf{b}$ is projected onto a line $\mathbf{a}$, its projection $\mathbf{p}$ is the part of $\mathbf{b}$ along that line $\mathbf{a}$.

Let's look at interactive graphic (3.4) for [section 3.2.2: Projections](http://immersivemath.com/ila/ch03_dotproduct/ch03.html) of the [Immersive Linear Algebra online book](http://immersivemath.com/ila/index.html).

<img src="images/projection_line.png" alt="projection" style="width: 70%"/>
(source: [Immersive Math](http://immersivemath.com/ila/ch03_dotproduct/ch03.html))

And here is what it looks like to project a vector onto a plane:

<img src="images/projection.png" alt="projection" style="width: 70%"/>
(source: [The Linear Algebra View of Least-Squares Regression](https://medium.com/@andrew.chamberlain/the-linear-algebra-view-of-least-squares-regression-f67044b7f39b))

When vector $\mathbf{b}$ is projected onto a line $\mathbf{a}$, its projection $\mathbf{p}$ is the part of $\mathbf{b}$ along that line $\mathbf{a}$.  So $\mathbf{p}$ is some multiple of $\mathbf{a}$. Let $\mathbf{p} = \hat{x}\mathbf{a}$ where $\hat{x}$ is a scalar.

#### Orthogonality

**The key to projection is orthogonality:** The line *from* $\mathbf{b}$ to $\mathbf{p}$ (which can be written $\mathbf{b} - \hat{x}\mathbf{a}$) is perpendicular to $\mathbf{a}$.

This means that $$ \mathbf{a} \cdot (\mathbf{b} -  \hat{x}\mathbf{a}) = 0 $$

and so $$\hat{x} = \frac{\mathbf{a} \cdot \mathbf{b}}{\mathbf{a} \cdot \mathbf{a}} $$

### The Algorithm

**Motivation**:

We want orthonormal columns in $Q$ and a Hessenberg $H$ such that $A Q = Q H$.

Thinking about it iteratively, $$ A Q_n = Q_{n+1} H_n $$ where $Q_{n+1}$ is $n\times n+1$ and $H_n$ is $n+1 \times n$.  This creates a solvable recurrence relation.

<img src="images/arnoldi.jpg" alt="arnoldi" style="width: 95%"/>
(source: Trefethen, Lecture 33)

**Pseudo-code for Arnoldi Algorithm**

    Start with an arbitrary vector (normalized to have norm 1) for first col of Q
    for n=1,2,3...
        v = A @ nth col of Q
        for j=1,...n
            project v onto q_j, and subtract the projection off of v
            want to capture part of v that isn't already spanned by prev columns of Q
            store coefficients in H
        normalize v, and then make it the (n+1)th column of Q

Notice that we are multiplying A by the previous vector in Q and removing the components that are not orthogonal to the existing columns of Q.

**Question:** Repeated multiplications of A?  Does this remind you of anything?

#### Answer:

In [ ]:
#Exercise Answer
The *Power Method* involved iterative multiplications by A as well!  

### About how the Arnoldi Iteration works

- With the Arnoldi Iteration, we are finding an orthonormal basis for the *Krylov subspace*. 
The Krylov matrix $$ K = \left[b \; Ab \; A^2b \; \dots \; A^{n-1}b \right]$$
has a QR factorization
$$K = QR$$
and that is the same $Q$ that is being found in the Arnoldi Iteration.  Note that the Arnoldi Iteration does not explicity calculate $K$ or $R$.

- Intuition: K contains good information about the largest eigenvalues of A, and the QR factorization reveals this information by peeling off one approximate eigenvector at a time.


The Arnoldi Iteration is two things:
1. the basis of many of the iterative algorithms of numerical linear algebra
2. a technique for finding eigenvalues of nonhermitian matrices
(Trefethen, page 257)

**How Arnoldi Locates Eigenvalues**

1. Carry out Arnoldi iteration
2. Periodically calculate the eigenvalues (called *Arnoldi estimates* or *Ritz values*) of the Hessenberg H, using the QR algorithm
3. Check at whether these values are converging.  If they are, they're probably eigenvalues of A.

### Implementation

In [246]:
# Decompose square matrix A @ Q ~= Q @ H
def arnoldi(A):
    m, n = A.shape
    assert(n <= m)
    
    # Hessenberg matrix
    H = np.zeros([n+1,n]) #, dtype=np.float64)
    # Orthonormal columns
    Q = np.zeros([m,n+1]) #, dtype=np.float64)
    # 1st col of Q is a random column with unit norm
    b = np.random.rand(m)
    Q[:,0] = b / np.linalg.norm(b)
    for j in range(n):
        v = A @ Q[:,j]
        for i in range(j+1):
            #This comes from the formula for projection of v onto q.
            #Since columns q are orthonormal, q dot q = 1
            H[i,j] = np.dot(Q[:,i], v)
            v = v - (H[i,j] * Q[:,i])
        H[j+1,j] = np.linalg.norm(v)
        Q[:,j+1] = v / H[j+1,j]
        
        # printing this to see convergence, would be slow to use in practice
        print(np.linalg.norm(A @ Q[:,:-1] - Q @ H))
    return Q[:,:-1], H[:-1,:]

In [86]:
Q, H = arnoldi(A)

8.59112969133
4.45398729097
0.935693639985
3.36613943339
0.817740180293


Check that H is tri-diagonal:

In [70]:
H

array([[ 5.62746,  4.05085, -0.     ,  0.     , -0.     ],
       [ 4.05085,  3.07109,  0.33036,  0.     , -0.     ],
       [ 0.     ,  0.33036,  0.98297,  0.11172, -0.     ],
       [ 0.     ,  0.     ,  0.11172,  0.29777,  0.07923],
       [ 0.     ,  0.     ,  0.     ,  0.07923,  0.06034]])

#### Exercise

Write code to confirm that:
1. AQ = QH
2. Q is orthonormal

#### Answer:

In [71]:
#Exercise:
np.allclose(A @ Q, Q @ H)

True

In [456]:
#Exercise:
np.allclose(np.eye(len(Q)), Q.T @ Q)

True

#### General Case:

**General Matrix**: Now we can do this on our general matrix A (not symmetric).  In this case, we are getting a Hessenberg instead of a Tri-diagonal

In [74]:
Q0, H0 = arnoldi(A0)

1.44287067354
1.06234006889
0.689291414367
0.918098818651
4.7124490411e-16


Check that H is Hessenberg:

In [76]:
H0

array([[ 1.67416,  0.83233, -0.39284,  0.10833,  0.63853],
       [ 1.64571,  1.16678, -0.54779,  0.50529,  0.28515],
       [ 0.     ,  0.16654, -0.22314,  0.08577, -0.02334],
       [ 0.     ,  0.     ,  0.79017,  0.11732,  0.58978],
       [ 0.     ,  0.     ,  0.     ,  0.43238, -0.07413]])

In [77]:
np.allclose(A0 @ Q0, Q0 @ H0)

True

In [78]:
np.allclose(np.eye(len(Q0)), Q0.T @ Q0), np.allclose(np.eye(len(Q0)), Q0 @ Q0.T)

(True, True)

## Putting it all together

In [215]:
def eigen(A, max_iter=20):
    Q, H = arnoldi(A)
    Ak, QQ = practical_qr(H, max_iter)
    U = Q @ QQ
    D = np.diag(Ak)
    return U, D

In [213]:
n = 10
A0 = np.random.rand(n,n)
A = A0 @ A0.T

In [242]:
U, D = eigen(A, 40)

14.897422908
1.57451192745
1.4820012435
0.668164424736
0.438450319682
0.674050723258
1.19470880942
0.217103444634
0.105443975462
3.8162597576e-15
[ 27.34799   1.22613   1.29671   0.70253   0.49651   0.56779   0.60974
   0.70123   0.19209   0.04905]
[ 27.34981   1.85544   1.04793   0.49607   0.44505   0.7106    1.00724
   0.07293   0.16058   0.04411]
[ 27.34981   2.01074   0.96045   0.54926   0.61117   0.8972    0.53424
   0.19564   0.03712   0.04414]
[ 27.34981   2.04342   0.94444   0.61517   0.89717   0.80888   0.25402
   0.19737   0.03535   0.04414]
[ 27.34981   2.04998   0.94362   0.72142   1.04674   0.58643   0.21495
   0.19735   0.03534   0.04414]
[ 27.34981   2.05129   0.94496   0.90506   0.95536   0.49632   0.21015
   0.19732   0.03534   0.04414]
[ 27.34981   2.05156   0.94657   1.09452   0.79382   0.46723   0.20948
   0.1973    0.03534   0.04414]
[ 27.34981   2.05161   0.94863   1.1919    0.70539   0.45628   0.20939
   0.19728   0.03534   0.04414]
[ 27.34981   2.05162   0.95178

In [240]:
D

array([ 5.10503,  0.58805,  0.49071, -0.65174, -0.60231, -0.37664,
       -0.13165,  0.0778 , -0.10469, -0.29771])

In [241]:
np.linalg.eigvals(A)

array([ 27.34981,   2.05162,   1.24292,   0.94505,   0.66585,   0.44839,
         0.20948,   0.19717,   0.04414,   0.03534])

In [236]:
np.linalg.norm(U @ np.diag(D) @ U.T - A)

0.0008321887107978883

In [237]:
np.allclose(U @ np.diag(D) @ U.T, A, atol=1e-3)

True

### Further Reading

Let's find some eigenvalues!


from [Nonsymmetric Eigenvalue Problems](https://sites.math.washington.edu/~morrow/498_13/eigenvalues.pdf) chapter:

Note that "direct" methods must still iterate, since finding eigenvalues is mathematically equivalent to finding zeros of polynomials, for which no noniterative methods can exist. We call a method direct if experience shows that it (nearly) never fails to converge in a
fixed number of iterations.

Iterative methods typically provide approximations only to a subset of the eigenvalues and eigenvectors and are usually run only long enough to get a few adequately accurate eigenvalues rather than a large number

our ultimate algorithm: the shifted Hessenberg QR algorithm

More reading:
- [The Symmetric Eigenproblem and SVD](https://sites.math.washington.edu/~morrow/498_13/eigenvalues2.pdf)
- [Iterative Methods for Eigenvalue Problems](https://sites.math.washington.edu/~morrow/498_13/eigenvalues3.pdf) Rayleigh-Ritz Method, Lanczos algorithm

### Coming Up

We will be coding our own QR decomposition (two different ways!) in the future, but first we are going to see another way that the QR decomposition can be used: to calculate linear regression.

## End

### Miscellaneous Notes

Symmetric matrices come up naturally:
- distance matrices
- relationship matrices (Facebook or LinkedIn)
- ODEs

We will look at positive definite matrices, since that guarantees that all the eigenvalues are real.

Note: in the confusing language of NLA, the QR algorithm is *direct*, because you are making progress on all columns at once.  In other math/CS language, the QR algorithm is *iterative*, because it iteratively converges and never reaches an exact solution.

structured orthogonalization.  In the language of NLA, Arnoldi iteration is considered an *iterative* algorithm, because you could stop part way and have a few columns completed.

a Gram-Schmidt style iteration for transforming a matrix to Hessenberg form